In [347]:
import numpy as np
import read_data
import math

In [348]:
train_data = read_data.preprocess("Dataset/DT/dtree_data/train.csv")
valid_data = read_data.preprocess("Dataset/DT/dtree_data/valid.csv")
test_data = read_data.preprocess("Dataset/DT/dtree_data/test.csv")

In [349]:
print (train_data.shape)
print (valid_data.shape)
print (test_data.shape)
type(train_data.shape)

(27000, 15)
(3000, 15)
(7000, 15)


tuple

In [350]:
class Node:
    
    def __init__(self):
        self.splitAttribute=-1
        self.pred=-1
        self.positives=0
        self.negatives=0
        self.children={}
        self.median=-1
        self.isNumeric=False

In [351]:
def growTree(data,label,hasBeenSplit,isNumeric):
    
    global nodes
    nodes+=1
#     get Best Splitting Attribute
    splitAttribute=getSplitAttribute(data,label,hasBeenSplit,isNumeric)
    
    instances=len(data)
    positives=0.0
    negatives=0.0
    
    for i in range(instances):
        if(label[i]==1):
            positives+=1
        else:
            negatives+=1
#     Declaring and Initialising Root
    root=Node()
    root.splitAttribute=splitAttribute
    root.positives=positives
    root.negatives=negatives
    if(positives>=negatives):
        root.pred=1
    else:
        root.pred=0
        
    if(splitAttribute==-1):
        return root
    
    hasBeenSplit[splitAttribute]=True
    distributeData={}
    distributeLabel={}
    
    if(isNumeric[splitAttribute]):
        root.isNumeric=True
        print("NUmeric")
        pass
    else:
        for i in range(instances):
            key=data[i][splitAttribute]
            
            if key in distributeData:
                distributeData[key].append(data[i])
                distributeLabel[key].append(label[i])
            
            else:
                distributeData[key]=[data[i]]
                distributeLabel[key]=[label[i]]
    
    for key,value in distributeData.items():
        root.children[key]=growTree(value,distributeLabel[key],hasBeenSplit,isNumeric)
        
    hasBeenSplit[splitAttribute]=False
    return root

In [352]:
def getSplitAttribute(data,label,hasBeenSplit,isNumeric):
    attributes=len(hasBeenSplit)
    
    curEntropy=Hy(label)
    
    bestAttribute=-1
    conditionalEntropy=1e9
    instances=len(data)
        
    for i in range(attributes):
        if(isNumeric[i]):
            print("HI")
            pass
        if(hasBeenSplit[i]):
#             print("Splitted")
            continue
        counterTable={}
#         print(i)
        
        for j in range(instances):
            key=data[j][i]
            
            if key in counterTable:
                counterTable[key][label[j]]+=1
            else:
                counterTable[key]=[0,0]
#                 print (counterTable)
                counterTable[key][label[j]]+=1
        
        entropy=0
        for key,value in counterTable.items():
            prob=(value[0]+value[1])/float(instances)
            entropy+=prob*getEntropy(value[1],value[0])
        
#         print(entropy)
        if(entropy<conditionalEntropy):
            conditionalEntropy=entropy
            bestAttribute=i
    
#     print(curEntropy)
#     print(conditionalEntropy)
    
    informationGain=curEntropy-conditionalEntropy
    
    if(informationGain<=0):
        return -1
    
    return bestAttribute
            
                
        
        

In [353]:
def Hy(y):
    instances=len(y)
    positives=0
    negatives=0
    
    for i in range(instances):
        if(y[i]==0):
            negatives+=1
        else:
            positives+=1

    return getEntropy(positives,negatives)

In [354]:
def getEntropy(positives,negatives):
    p1=positives/float(positives+negatives)
    p0=negatives/float(positives+negatives)
    
    entropy=0
    
    if(negatives!=0):
        entropy+=(-p0*math.log(p0))
    
    if(positives!=0):
        entropy+=(-p1*math.log(p1))
    
    return entropy

In [355]:
def getPrediction(root,instance):
    if(root.splitAttribute==-1):
        return root.pred
    
    if(root.isNumeric):
        return 0
    else:
        key=instance[root.splitAttribute]
        if key in root.children:
            return getPrediction(root.children[key],instance)
        else:
            return self.pred
    


In [356]:
def getAccuracy(root,data,label):
    
    instance=len(data)
    correct=0
    
    for i in range(instance):
        if(label[i]==getPrediction(root,data[i])):
            correct+=1
    
    return correct/float(instance)

In [357]:
getEntropy(1,4)

0.5004024235381879

In [358]:
# test_data

In [359]:
(instances,attributes)=train_data.shape
trainLabel=train_data[:,0]
trainData=train_data[:,1:attributes]
print (trainLabel.shape)
print (testLabel.shape)
hasBeenSplit=[]
isNumeric=[]

for i in range(attributes-1):
    hasBeenSplit.append(False)
    isNumeric.append(False)

trainDataList=trainData.tolist()
trainLabelList=trainLabel.tolist()
# trainLabelList=zip(*trainLabelList)
# trainLabelList[0]
# print(len(trainDataList))
# print(len(trainLabelList[0]))

(27000,)
(27000, 14)


In [360]:
nodes=0
root=growTree(trainDataList,trainLabelList,hasBeenSplit,isNumeric)

In [363]:
print (nodes)
print (getAccuracy(root,trainDataList,trainLabelList))


7902
0.8893333333333333


In [362]:
# b=np.array([1,2,3])
# b.shape
# type((b.tolist())[0])